_Does size matter? The effect of Instagram influencer account size on post sentiment and resulting marketing outcomes_

_Master's thesis by Thomas A. Frost_

# Part 0: SocialBlade Scraper

It is necessary to download the SocialBlade pages as html files and save them in the corresponding folder

## Library Installs

## Loading

## Functions

## Here comes the code...
